In [1]:
%load_ext autoreload
%autoreload 2

# Tagging

The tagging chain uses the OpenAI `functions` parameter to specify a schema to tag a document with.

In [4]:
from langchain.chains.openai_functions import create_tagging_chain_pydantic

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain
from langchain.prompts import ChatPromptTemplate

In [6]:
llm = ChatOpenAI(temperature=0, 
                   model="gpt-3.5-turbo-0613")

## Simplest approach, only specifying type

In [7]:
schema = {
      "properties": {
          "sentiment": {"type": "string"}, 
          "aggressiveness": {"type": "integer"},
          "language": {"type": "string"},
              }
    }

In [8]:
chain = create_tagging_chain(schema, llm)

In [59]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': 'positive', 'language': 'Spanish'}

In [60]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': 'enojado', 'aggressiveness': 1, 'language': 'Spanish'}

In [61]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.run(inp)

{'sentiment': 'positive', 'aggressiveness': 0, 'language': 'English'}

## More control
- possible values for each property
- description to make sure that the model understands the property
- required properties to be returned

In [43]:
schema = {
      "properties": {
          "sentiment": {"type": "string", "enum": ["happy", "neutral", "sad"]}, 
          "aggressiveness": {"type": "integer", "enum": [1,2,3,4,5], "description": "describes how aggressive the statement is, the higher the number the more aggressive"},
          "language": {"type": "string", "enum": ["spanish", "english", "french", "german", "italian"]},
              },
        "required": ["language", "sentiment", "aggressiveness"]
    }

In [44]:
chain = create_tagging_chain(schema, llm)

In [99]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': 'happy', 'aggressiveness': 0, 'language': 'spanish'}

In [100]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': 'sad', 'aggressiveness': 10, 'language': 'spanish'}

In [101]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.run(inp)

{'sentiment': 'neutral', 'aggressiveness': 0, 'language': 'english'}

In [10]:
from pydantic import BaseModel
import json

In [56]:
from enum import Enum
from pydantic import BaseModel, Field

class PydanticSchema(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(..., description="describes how aggressive the statement is, the higher the number the more aggressive", enum=[1, 2, 3, 4, 5])
    language: str = Field(..., enum=["spanish", "english", "french", "german", "italian"])

    class Config:
        use_enum_values = True

In [58]:
chain = create_tagging_chain_pydantic(MyModel, llm)

{'title': 'MyModel', 'type': 'object', 'properties': {'sentiment': {'title': 'Sentiment', 'enum': ['happy', 'neutral', 'sad'], 'type': 'string'}, 'aggressiveness': {'title': 'Aggressiveness', 'description': 'describes how aggressive the statement is, the higher the number the more aggressive', 'enum': [1, 2, 3, 4, 5], 'type': 'integer'}, 'language': {'title': 'Language', 'enum': ['spanish', 'english', 'french', 'german', 'italian'], 'type': 'string'}}, 'required': ['sentiment', 'aggressiveness', 'language']}


In [59]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.run(inp)

In [60]:
res

MyModel(sentiment='sad', aggressiveness=10, language='spanish')